In [1]:
!pip install sqlalchemy

In [2]:
import pandas as pd
import numpy as np
import pymysql 
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [5]:
pip install mysql-connector-python

     |████████████████████████████████| 565 kB 4.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import mysql.connector as mp

cnx = mp.connect(
    host='localhost',  # replace with your host name
    user='root',  # replace with your username
    password='2735272Rodj',  # replace with your password
    database='sakila'  # replace with your database name
)
# Check if the connection is established
if cnx.is_connected():
    print("Connected to the database")
else:
    print("Failed to connect to the database")

# Don't forget to close the connection


Connected to the database


In [13]:
import pandas as pd

def rentals_month(cnx, month, year):
    query = """
    SELECT *
    FROM rental
    WHERE MONTH(rental_date) = %s AND YEAR(rental_date) = %s
    """
    df = pd.read_sql_query(query, cnx, params=(month, year))
    return df
print(df)

      rental_id         rental_date  inventory_id  customer_id  \
0             1 2005-05-24 22:53:30           367          130   
1             2 2005-05-24 22:54:33          1525          459   
2             3 2005-05-24 23:03:39          1711          408   
3             4 2005-05-24 23:04:41          2452          333   
4             5 2005-05-24 23:05:21          2079          222   
...         ...                 ...           ...          ...   
1151       1153 2005-05-31 21:36:44          2725          506   
1152       1154 2005-05-31 21:42:09          2732           59   
1153       1155 2005-05-31 22:17:11          2048          251   
1154       1156 2005-05-31 22:37:34           460          106   
1155       1157 2005-05-31 22:47:45          1449           61   

             return_date  staff_id         last_update  
0    2005-05-26 22:04:30         1 2006-02-15 21:30:53  
1    2005-05-28 19:40:33         1 2006-02-15 21:30:53  
2    2005-06-01 22:12:39         1 2

In [11]:
def rental_count_month(df, month, year):
    # Group by customer_id and count the number of rentals
    df_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')
    return df_count
df = rentals_month(cnx, 5, 2005)  # Get rentals for May 2005
df_count = rental_count_month(df, 5, 2005)  # Count rentals by customer for May 2005
print(df_count)

     customer_id  rentals_5_2005
0              1               2
1              2               1
2              3               2
3              5               3
4              6               3
..           ...             ...
515          594               4
516          595               1
517          596               6
518          597               2
519          599               1

[520 rows x 2 columns]


In [12]:
def compare_rentals(df1, df2):
    # Merge the two DataFrames on customer_id
    df = pd.merge(df1, df2, on='customer_id', suffixes=('_month1', '_month2'))

    # Calculate the difference between the number of rentals in the two months
    df['difference'] = df.iloc[:, 1] - df.iloc[:, 2]

    return df
df1 = rental_count_month(rentals_month(cnx, 5, 2005), 5, 2005)  # Count rentals by customer for May 2005
df2 = rental_count_month(rentals_month(cnx, 6, 2005), 6, 2005)  # Count rentals by customer for June 2005

df_compare = compare_rentals(df1, df2)  # Compare rentals in May and June 2005
print(df_compare)

     customer_id  rentals_5_2005  rentals_6_2005  difference
0              1               2               7          -5
1              2               1               1           0
2              3               2               4          -2
3              5               3               5          -2
4              6               3               4          -1
..           ...             ...             ...         ...
507          594               4               6          -2
508          595               1               2          -1
509          596               6               2           4
510          597               2               3          -1
511          599               1               4          -3

[512 rows x 4 columns]


/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
